# Hyperparameter Optimization for Recent Stations

In [ ]:
import sys
import os
# Add the parent directory to the Python path
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
)

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np

import importlib
import utils
import modelsRecent

In [ ]:
# In case of 'modelsRecent.py' modifications
importlib.reload(modelsRecent)

In [ ]:
def filter_stations(data_dict, stations_to_include):
    return {
        station: df for station, df in data_dict.items() 
        if station in stations_to_include
    }

In [ ]:
RECENT_STATIONS = ['P6E', 'BDC', 'W80'] # W14 and QD6 are treated in another notebook

In [ ]:
# Load Data
x_train = x_train = pd.read_csv('train_f_x.csv')
y_train = pd.read_csv('y_train_sncf.csv')

In [ ]:
# Data Preparation
df = utils.prepare_backtest_data(x_train, y_train, remove_covid=True)

# Extract recent stations data
df_recent = filter_stations(df, RECENT_STATIONS)

# Split into train and test dataset
df_train = {}
df_test = {}
for station in df_recent:
    df_train_station, df_test_station = utils.split_dataset(df_recent[station], cut_date='2022-09-17')
    df_train[station] = df_train_station
    df_test[station] = df_test_station

In [ ]:
# Keep true values 
df_test_true = {
    station: df_test[station].copy()
    for station in df_test.keys()
}

In [ ]:
# imports
import optuna
import tqdm
from optuna.samplers import TPESampler
import copy

In [ ]:
def objectiveRNN(trial):
    """
    Optuna objective function to minimize the average MAPE score 
    across the stations.
    """
    # Define the search space for hyperparameters
    params = {
        "units": trial.suggest_int("units", 30, 60, step=5),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [15, 20, 25, 30, 35, 40]),
        "seq_len": trial.suggest_int("seq_len", 10, 40, step=5)
    }

    trial_sample_test = copy.deepcopy(df_test)

    try:
        # 3. Call backtest_lstm with suggested parameters
        # We ignore the returned df and losses to save memory during optimization
        _, mape_results, _ = modelsRecent.backtest_model(
            df_train, 
            trial_sample_test, 
            df_test_true, 
            seq_len=params["seq_len"],
            units=params["units"],
            activation='tanh',
            learning_rate=params["learning_rate"],
            batch_size=params["batch_size"],
            epochs=100, 
            keep_percentage=0.5,
            early_stop=True, 
            features=['job', 'ferie', 'vacances'],
            architecture = 'rnn'
        )

        # 4. Handle failed trials within the backtest
        if not mape_results:
            return float('inf')

        # 5. Calculate the MEAN MAPE across all stations in the sample
        # This makes the hyperparameters generalize better across different stations
        all_mapes = [res['MAPE'] for res in mape_results]
        average_mape = np.mean(all_mapes)
        
        return average_mape

    except Exception as e:
        print(f"Trial failed with error: {e}")
        return float('inf')

In [ ]:
def objectiveLSTM(trial):
    """
    Optuna objective function to minimize the average MAPE score 
    across the stations.
    """
    # Define the search space for hyperparameters
    params = {
        "units": trial.suggest_int("units", 30, 60, step=5),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [15, 20, 25, 30, 35, 40]),
        "seq_len": trial.suggest_int("seq_len", 10, 40, step=5)
    }

    trial_sample_test = copy.deepcopy(df_test)

    try:
        # 3. Call backtest_lstm with suggested parameters
        # We ignore the returned df and losses to save memory during optimization
        _, mape_results, _ = modelsRecent.backtest_model(
            df_train, 
            trial_sample_test, 
            df_test_true, 
            seq_len=params["seq_len"],
            units=params["units"],
            activation='tanh',
            learning_rate=params["learning_rate"],
            batch_size=params["batch_size"],
            epochs=100, 
            keep_percentage=0.5,
            early_stop=True, 
            features=['job', 'ferie', 'vacances'],
            architecture = 'lstm'
        )

        # 4. Handle failed trials within the backtest
        if not mape_results:
            return float('inf')

        # 5. Calculate the MEAN MAPE across all stations in the sample
        # This makes the hyperparameters generalize better across different stations
        all_mapes = [res['MAPE'] for res in mape_results]
        average_mape = np.mean(all_mapes)
        
        return average_mape

    except Exception as e:
        print(f"Trial failed with error: {e}")
        return float('inf')

In [ ]:
# Run flag
run_flag = 0

In [ ]:
if (run_flag == 1):
    # Create and run the study
    studyRNN = optuna.create_study(direction="minimize")
    studyRNN.optimize(objectiveRNN, n_trials=35)
    print("Optimization Finished!")

    # Increment the run flag
    run_flag+=1

    # Display results
    print(f"Best MAPE: {studyRNN.best_value:.4f}")
    print("Best Hyperparameters for RNN:", studyRNN.best_params)

In [ ]:
if (run_flag == 2):
    # Create and run the study
    studyLSTM = optuna.create_study(direction="minimize")
    studyLSTM.optimize(objectiveLSTM, n_trials=35)
    print("Optimization Finished!")

    # Increment the run flag
    run_flag+=1

    # Display results
    print(f"Best MAPE: {studyLSTM.best_value:.4f}")
    print("Best Hyperparameters for LSTM:", studyLSTM.best_params)

In [ ]:
# Best Hyperparameters for RNN: {'units': 50, 'learning_rate': 0.004511134598760262, 'batch_size': 15, 'seq_len': 25}
# Best MAPE: 0.3443
#
# Best Hyperparameters for LSTM: {'units': 50, 'learning_rate': 0.00010728642971712255, 'batch_size': 15, 'seq_len': 25}
# Best MAPE: 0.4241